In [1]:
import numpy as np

In [146]:
mu_earth = 398600.4418  # km^3/s^2
mu_moon = 4902.8  # km^3/s^2
T = 27.32 * 24 * 60 * 60  # in seconds
R = 384400  # km

def vector_moon(t):
    return np.array([R*np.cos(2*np.pi/T*t), R*np.sin(2*np.pi/T*t), 0])

def cislunar_gravity(r_craft, t):
    r_craft_moon = r_craft - vector_moon(t)
    moon_influence = -(r_craft_moon * mu_moon) / np.linalg.norm(r_craft_moon)**3
    earth_influence = -r_craft * mu_earth / np.linalg.norm(r_craft)**3
    return moon_influence + earth_influence

def degree_angle(A, B) -> float:
    """Double check"""
    cos_theta = np.dot(A,B) / (np.linalg.norm(A) * np.linalg.norm(B))
    angle = np.arccos(cos_theta)
    return np.degrees(angle)

def is_visible(r_sensor, r_target) -> bool:
    # constant
    max_range = 2000

    distance = np.linalg.norm(r_sensor - r_target)
    angle = degree_angle(r_sensor, r_target)
    if distance > max_range or angle > 40:
        return False
    return True

def vector_sensor(t):
    r_sensor_moon = np.array([0, 0, -1736])
    return vector_moon(t) + r_sensor_moon


In [157]:
# initial state: set relative to Earth
pos_craft = np.array([325.59, 446.71, 1791.67]) + vector_moon(0)
v_craft = np.array([-1.78, 0.08, 0.3]) + (vector_moon(1) - vector_moon(0))

# simulation
history = []
for t in range(1, 86400):
    a_craft = cislunar_gravity(pos_craft, t)
    v_craft += a_craft  # update velocity
    pos_craft += v_craft # Update position
    pos_sensor = vector_sensor(t)
    pos_moon = vector_moon(t)
    vis = is_visible(pos_sensor, pos_craft)
    angle = degree_angle(pos_sensor, pos_craft)
    history.append({'moon': pos_moon,
                    'sensor': pos_sensor,
                    'craft': pos_craft,
                    'angle': angle,
                    'isVisible': vis,
                    'acceleration': a_craft,
                    'velocity': v_craft})
